In [1]:
#Imports
import requests
import pandas as pd
import pandasql as psql

In [2]:
#Url da Fake Store API
url = 'https://fakestoreapi.com/'

In [3]:
#Definindo a função que faz a chamada à API FakeStore, retornando o json com os resultados ou uma exceção.
def busca_dados(endpoint):
    try: 
        response = requests.get(f'{url}{endpoint}')
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Erro ao buscar dados do endpoint {endpoint}")
        return None

In [4]:
#No desenvolvimento do código notei que várias colunas vieram aninhadas "nested columns" e optei 
#por criar uma função que resolva o problema.
def trata_aninhamento(df, coluna):
    df_normalizado = pd.json_normalize(df[coluna])
    return df_normalizado

#### dProducts

In [8]:
#Recuperando os dados da dimensão "products"
df_products = pd.DataFrame(busca_dados('products'))

In [6]:
#Tratando a coluna aninhada 'rating'
df_products = df_products.drop(columns='rating') \
                         .join(trata_aninhamento(df_products,coluna_aninhada))

KeyError: "['rating'] not found in axis"

In [ ]:
#Separando somente as colunas que serão necessárias para análise
colunas = ['id','category']
df_products = df_products[colunas]

#### dUsers

In [ ]:
#Recuperando os dados da dimensão "users"
df_users = pd.DataFrame(busca_dados('users'))

In [ ]:
# Tratando as colunas aninhadas 'name' e 'address'
df_users = df_users.drop(columns=['name','address']) \
                   .join(trata_aninhamento(df_users,'name')) \
                   .join(trata_aninhamento(df_users,'address'))

In [ ]:
#Separando as colunas que serão necessárias para análise
cols = ['id','username','firstname','lastname']
df_users = df_users[cols]

#### fCart

In [ ]:
#Recuperando os dados da fato Cart
df_cart = pd.DataFrame(busca_dados('carts'))

In [ ]:
#O DataFrame df_cart possui colunas com listas de objetos, portanto é necessário "explodir" estes objetos em várias linhas
df_cart_exploded = df_cart.explode('products')

In [ ]:
#Tratando a coluna aninhada "products" 
df_cart = df_cart_exploded.drop(columns=['products']).join(trata_aninhamento(df_cart_exploded,'products'))

In [ ]:
#Para criar o DataFrame final que responde as perguntas de negócio do Case, optei por usar SQL. A biblioteca pandasql permite 
#trabalhar com SQL na manipulação de DataFrames utilizando as funções do sqlite, apesar de não apresentar a performance ideal 
#para bases muito grandes, neste caso será o suficiente.

query = """
    SELECT
        u.id as user_id,
        u.firstname || ' ' || u.lastname as user_name,
        MAX(c.date) as date_last_addition,
        p.category,
        SUM(c.quantity)
    FROM df_users u
        LEFT JOIN df_cart c ON u.id = c.userId
        LEFT JOIN df_products p ON c.productId = p.id
    GROUP BY 
        u.id
"""
df_analise = psql.sqldf(query, locals())